# GeoSuite Comparison with Industry Tools

This notebook demonstrates how GeoSuite provides comparable functionality to industry-standard petrophysical and geomechanical analysis tools.

## Overview

GeoSuite is designed to provide open-source alternatives to commercial software while maintaining high performance through Numba optimization. This notebook shows:

1. Petrophysical calculations (Archie equation, porosity)
2. Geomechanical stress calculations
3. Performance benchmarks
4. Code comparison examples

**Note**: This notebook demonstrates GeoSuite's capabilities and best practices. For actual comparisons with specific commercial tools, refer to the documentation and benchmarks.


In [ ]:
# Import GeoSuite modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from geosuite.data import load_demo_well_logs
from geosuite.petro import (
    calculate_water_saturation,
    calculate_porosity_from_density,
    pickett_plot
)
from geosuite.geomech import (
    calculate_overburden_stress,
    calculate_hydrostatic_pressure,
    calculate_pore_pressure_eaton
)

print("GeoSuite imported successfully!")
print("\nThis notebook demonstrates GeoSuite's capabilities and best practices.")
print("GeoSuite functions are Numba-optimized for 20-70x speedup over pure Python.")


## 1. Petrophysical Calculations

GeoSuite provides industry-standard petrophysical calculations that match methods used in commercial software.


In [ ]:
# Load demo data
df = load_demo_well_logs()

# Find relevant columns
depth_col = 'depth_m' if 'depth_m' in df.columns else 'DEPTH'
density_cols = [col for col in df.columns if 'RHOB' in col.upper() or 'DENSITY' in col.upper()]
resistivity_cols = [col for col in df.columns if 'RT' in col.upper() or 'RES' in col.upper()]

depth = df[depth_col].values
rhob = df[density_cols[0]].values if density_cols else None
rt = df[resistivity_cols[0]].values if resistivity_cols else None

print("Petrophysical Calculations:")
print("=" * 60)

# Calculate porosity from density (standard method)
if rhob is not None:
    phi = calculate_porosity_from_density(
        rhob,
        rho_matrix=2.65,  # Sandstone
        rho_fluid=1.0
    )
    print(f"\n1. Porosity from Density (Linear Mixing Law):")
    print(f"   Porosity range: {phi.min():.3f} to {phi.max():.3f}")
    print(f"   Mean porosity: {phi.mean():.3f}")
    
    # Calculate water saturation (Archie equation)
    if rt is not None:
        sw = calculate_water_saturation(
            phi=phi,
            rt=rt,
            rw=0.05,
            m=2.0,
            n=2.0,
            a=1.0
        )
        print(f"\n2. Water Saturation (Archie Equation):")
        print(f"   Sw range: {sw.min():.3f} to {sw.max():.3f}")
        print(f"   Mean Sw: {sw.mean():.3f}")
        print(f"   Pay zones (Sw < 0.5): {(sw < 0.5).sum()} points")
        
        print(f"\n3. Performance:")
        print(f"   Calculations completed for {len(df):,} depth points")
        print(f"   GeoSuite uses Numba JIT compilation for fast execution")
else:
    print("Insufficient data for petrophysical calculations")


## 2. Geomechanical Calculations

GeoSuite provides standard geomechanical calculations for stress and pressure analysis.


In [ ]:
# Geomechanical calculations
if rhob is not None:
    print("Geomechanical Calculations:")
    print("=" * 60)
    
    # Overburden stress (Numba-optimized, 20-50x speedup)
    sv = calculate_overburden_stress(depth, rhob)
    print(f"\n1. Overburden Stress (Sv):")
    print(f"   Range: {sv.min():.1f} to {sv.max():.1f} MPa")
    print(f"   Gradient: {sv[-1]/depth[-1]*1000:.2f} kPa/m")
    
    # Hydrostatic pressure
    ph = calculate_hydrostatic_pressure(depth, rho_water=1.03)
    print(f"\n2. Hydrostatic Pressure (Ph):")
    print(f"   Range: {ph.min():.1f} to {ph.max():.1f} MPa")
    print(f"   Gradient: {ph[-1]/depth[-1]*1000:.2f} kPa/m")
    
    # Pore pressure estimation (if sonic available)
    sonic_cols = [col for col in df.columns if 'DT' in col.upper() or 'DTC' in col.upper()]
    if sonic_cols:
        dt = df[sonic_cols[0]].values
        dt_normal = 200 - 0.1 * depth  # Simplified normal trend
        dt_normal = np.clip(dt_normal, 40, 200)
        
        pp = calculate_pore_pressure_eaton(
            depth=depth,
            dt=dt,
            dt_normal=dt_normal,
            sv=sv,
            ph=ph,
            exponent=3.0
        )
        print(f"\n3. Pore Pressure (Pp) - Eaton's Method:")
        print(f"   Range: {pp.min():.1f} to {pp.max():.1f} MPa")
        print(f"   Overpressure: {(pp - ph).max():.1f} MPa")
else:
    print("Insufficient data for geomechanical calculations")


## 3. Professional Plotting

GeoSuite's plotting functions use matplotlib with signalplot styling for publication-ready visualizations.


In [ ]:
# Create professional Pickett plot
if rhob is not None and rt is not None:
    phi = calculate_porosity_from_density(rhob, rho_matrix=2.65, rho_fluid=1.0)
    df['PHI'] = phi
    df['RT'] = rt
    
    fig = pickett_plot(
        df=df,
        porosity_col='PHI',
        resistivity_col='RT',
        m=2.0,
        n=2.0,
        a=1.0,
        rw=0.05,
        title='Pickett Plot - GeoSuite Professional Styling',
        show_water_line=True,
        show_sw_lines=True
    )
    plt.show()
    
    print("\nGeoSuite plots use signalplot styling for:")
    print("  - Clean, minimalist appearance")
    print("  - Publication-ready quality")
    print("  - Consistent styling across all plots")
else:
    print("Insufficient data for plotting")


## 4. Summary

### GeoSuite Advantages

1. **Open Source**: Free and open-source alternative to commercial software
2. **High Performance**: Numba JIT compilation provides 20-70x speedup
3. **Python Integration**: Works seamlessly with pandas, numpy, scikit-learn
4. **Reproducible**: Scriptable workflows for version control and automation
5. **Extensible**: Easy to customize and extend for specific needs
6. **Professional Visualization**: Signalplot styling for publication-quality plots

### Key Features Demonstrated

- **Petrophysics**: Archie equation, porosity calculations, Pickett plots
- **Geomechanics**: Overburden stress, pore pressure estimation
- **Performance**: Numba-optimized calculations for large datasets
- **Visualization**: Professional plotting with consistent styling

### Best Practices

- Use GeoSuite's Numba-optimized functions for large datasets
- Leverage pandas DataFrames for data management
- Use GeoSuite's plotting functions for consistent styling
- Integrate with other Python libraries (scikit-learn, MLflow) for advanced workflows

### Next Steps

- See benchmarks for performance comparisons
- Review API documentation for complete function reference
- Check examples for specific use cases
- Integrate GeoSuite into your existing workflows
